In [6]:
import cv2
import mediapipe as mp
import numpy as np
import time
import tensorflow as tf

In [7]:
mp_drawing = mp.solutions.drawing_utils
mp_facemesh = mp.solutions.face_mesh.FaceMesh(min_detection_confidence = 0.5,min_tracking_confidence=0.5)

In [9]:
def headpose(results,im_h,im_w,im_c):
    [face_3d,face_2d] = [[],[]]
    if(results.multi_face_landmarks):
        for face_landmarks in results.multi_face_landmarks:
            for idx,lm in enumerate(face_landmarks.landmark):
                if(idx in [33,263,1,61,291,199]):
                    if(idx==1):
                        nose_2d = (lm.x*im_w,lm.y*im_h)
                        nose_3d = (lm.x*im_w,lm.y*im_h,lm.z*3000)
                    x,y = int(lm.x*im_w),int(lm.y*im_h)
                    face_2d.append([x,y])
                    face_3d.append([x,y,lm.z])
        face_2d = np.array(face_2d).astype("float32")
        face_3d = np.array(face_3d).astype("float32")
        focal_length = 1 * im_w
    
        cam_mat = np.array([[focal_length,0,im_h/2],[0,focal_length,im_w/2],[0,0,1]])
        dist_mat = np.zeros((4,1),dtype = np.float32)
    
        suc,rot_v,tran_v = cv2.solvePnP(face_3d,face_2d,cam_mat,dist_mat)
        rmat,jac = cv2.Rodrigues(rot_v)
    
        angles,mtxR,mtxQ,Qx,Qy,Qz = cv2.RQDecomp3x3(rmat)
        x = angles[0]*360
        y = angles[1]*360
        z = angles[2]*360
    
        if(y<-10): text = "Looking right"
        elif(y>10) : text = "Looking left"
        elif(x<-13) : text = "Looking down"
        elif(x>13) : text = "Looking up"
        else : text = "Forward"
    else: text = "No Face Detected"
    return text

In [10]:
t=0
cap = cv2.VideoCapture(0)
while cap.isOpened(): 
    t += 1
    ret, frame = cap.read()
    im_h,im_w,im_c = frame.shape
    image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    results = mp_facemesh.process(image)
    # if(t%50==0):
    text = headpose(results,im_h,im_w,im_c)
    if(text == "Forward"):
        image = cv2.resize(image,(224,224))
        image = np.expand_dims(image, axis=0)
        image = image/255
        output = model.predict(image)
        print(output)
        predicted_class=np.argmax(output)
        cv2.putText(frame, label[predicted_class], (400, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    else:
        # discard
        pass
    # else : pass
    cv2.putText(frame,text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Head Pose Estimation",frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 2s 2s/step
[[0.00112732 0.9988727 ]]
1/1 [==============================] - 0s 69ms/step
[[0.0021468 0.9978532]]
1/1 [==============================] - 0s 87ms/step
[[0.01594395 0.984056  ]]
1/1 [==============================] - 0s 81ms/step
[[0.22709008 0.77290994]]
1/1 [==============================] - 0s 74ms/step
[[0.19927865 0.80072135]]
1/1 [==============================] - 0s 69ms/step
[[0.12354732 0.8764526 ]]
1/1 [==============================] - 0s 74ms/step
[[0.30458108 0.6954189 ]]
1/1 [==============================] - 0s 71ms/step
[[0.2234207 0.7765793]]
1/1 [==============================] - 0s 69ms/step
[[0.21694668 0.7830533 ]]
1/1 [==============================] - 0s 76ms/step
[[0.19637707 0.8036229 ]]
1/1 [==============================] - 0s 71ms/step
[[0.44858795 0.5514121 ]]
1/1 [==============================] - 0s 69ms/step
[[0.24274941 0.7572506 ]]
1/1 [==============================] - 0s 71ms/step
[[0.36004472 0.639